In [21]:
import numpy 
import sys 
import nmslib 
import time 
import math 
from sklearn.neighbors import NearestNeighbors
from sklearn.model_selection import train_test_split

In [22]:
# Just read the data
all_data_matrix = numpy.loadtxt('../../sample_data/sift_10k.txt')

In [23]:
# Create a held-out query data set
(data_matrix, query_matrix) = train_test_split(all_data_matrix, test_size = 0.1)

In [24]:
print("# of queries %d, # of data points %d"  % (query_matrix.shape[0], data_matrix.shape[0]) )

# of queries 1000, # of data points 9000


In [25]:
# Set index parameters
# These are the most important onese
M = 15
efC = 100

num_threads = 4
index_time_params = {'M': M, 'indexThreadQty': num_threads, 'efConstruction': efC, 'post' : 0,
                     'skip_optimized_index' : 1 # using non-optimized index!
                    }

In [26]:
# Number of neighbors 
K=100

In [27]:
# Space name should correspond to the space name 
# used for brute-force search
space_name='l2sqr_sift'

In [28]:
# Intitialize the library, specify the space, the type of the vector and add data points 
# for SIFT data, we want DENSE_UINT8_VECTOR and distance type INT
index = nmslib.init(method='hnsw', 
                    space=space_name, 
                    data_type=nmslib.DataType.DENSE_UINT8_VECTOR, 
                    dtype=nmslib.DistType.INT) 

In [29]:
index.addDataPointBatch(data_matrix.astype(numpy.uint8)) 

9000

In [30]:
# Create an index
start = time.time()
index.createIndex(index_time_params) 
end = time.time() 
print('Index-time parameters', index_time_params)
print('Indexing time = %f' % (end-start))

Index-time parameters {'M': 15, 'skip_optimized_index': 1, 'efConstruction': 100, 'post': 0, 'indexThreadQty': 4}
Indexing time = 0.168367


In [31]:
# Setting query-time parameters
efS = 100
query_time_params = {'efSearch': efS}
print('Setting query-time parameters', query_time_params)
index.setQueryTimeParams(query_time_params)

Setting query-time parameters {'efSearch': 100}


In [32]:
# Querying
query_qty = query_matrix.shape[0]
start = time.time() 
nbrs = index.knnQueryBatch(query_matrix.astype(numpy.uint8), k = K, num_threads = num_threads)
end = time.time() 
print('kNN time total=%f (sec), per query=%f (sec), per query adjusted for thread number=%f (sec)' % 
      (end-start, float(end-start)/query_qty, num_threads*float(end-start)/query_qty)) 

kNN time total=0.027784 (sec), per query=0.000028 (sec), per query adjusted for thread number=0.000111 (sec)


In [33]:
# Computing gold-standard data 
print('Computing gold-standard data')

start = time.time()
sindx = NearestNeighbors(n_neighbors=K, metric='l2', algorithm='brute').fit(data_matrix)
end = time.time()

print('Brute-force preparation time %f' % (end - start))

start = time.time() 
gs = sindx.kneighbors(query_matrix)
end = time.time()

print('brute-force kNN time total=%f (sec), per query=%f (sec)' % 
      (end-start, float(end-start)/query_qty) )

Computing gold-standard data
Brute-force preparation time 0.001349
brute-force kNN time total=0.235281 (sec), per query=0.000235 (sec)


In [34]:
# Finally computing recall
recall=0.0
for i in range(0, query_qty):
  correct_set = set(gs[1][i])
  ret_set = set(nbrs[i][0])
  recall = recall + float(len(correct_set.intersection(ret_set))) / len(correct_set)
recall = recall / query_qty
print('kNN recall %f' % recall)

kNN recall 0.990110


In [35]:
# Save a meta index
index.saveIndex('dense_index_nonoptim.bin')

In [36]:
# Re-intitialize the library, specify the space, the type of the vector.
newIndex = nmslib.init(method='hnsw', 
                        space=space_name, 
                        data_type=nmslib.DataType.DENSE_UINT8_VECTOR, 
                        dtype=nmslib.DistType.INT) 

In [37]:
# For non-optimized indices or methods different from HNSW we DO need to re-add data points
newIndex.addDataPointBatch(data_matrix.astype(numpy.uint8)) 

9000

In [38]:
# Re-load the index and re-run queries
newIndex.loadIndex('dense_index_nonoptim.bin')

In [39]:
# Setting query-time parameters and querying
print('Setting query-time parameters', query_time_params)
newIndex.setQueryTimeParams(query_time_params)

query_qty = query_matrix.shape[0]
start = time.time() 
new_nbrs = newIndex.knnQueryBatch(query_matrix.astype(numpy.uint8), k = K, num_threads = num_threads)
end = time.time() 
print('kNN time total=%f (sec), per query=%f (sec), per query adjusted for thread number=%f (sec)' % 
      (end-start, float(end-start)/query_qty, num_threads*float(end-start)/query_qty)) 

Setting query-time parameters {'efSearch': 100}
kNN time total=0.021776 (sec), per query=0.000022 (sec), per query adjusted for thread number=0.000087 (sec)


In [40]:
# Finally computing recall for the new result set
recall=0.0
for i in range(0, query_qty):
  correct_set = set(gs[1][i])
  ret_set = set(new_nbrs[i][0])
  recall = recall + float(len(correct_set.intersection(ret_set))) / len(correct_set)
recall = recall / query_qty
print('kNN recall %f' % recall)

kNN recall 0.990110
